In [136]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from scipy import stats
from scipy.stats import norm, skew

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [137]:
raw_train = train.copy()
raw_test = test.copy()

train.drop('Id',axis=1,inplace=True)
test.drop('Id',axis=1,inplace=True)

In [138]:
train.drop( train[train['GrLivArea']>4500].index,inplace=True)

In [139]:
train['SalePrice'] = np.log1p( train['SalePrice'])

In [140]:
y = train['SalePrice']
all_data = pd.concat([train,test],ignore_index=True)
all_data.drop('SalePrice',axis=1,inplace=True)

In [141]:
all_data.shape

(2917, 79)

In [142]:
train.dtypes

MSSubClass         int64
MSZoning          object
LotFrontage      float64
LotArea            int64
Street            object
                  ...   
MoSold             int64
YrSold             int64
SaleType          object
SaleCondition     object
SalePrice        float64
Length: 80, dtype: object

In [143]:
all_data['LotFrontage'].value_counts().sort_values(ascending=False).values[0]

276

In [144]:
missing_count = all_data.isnull().sum()
missing_count = missing_count[missing_count > 0]
missing_percent = missing_count / len(all_data) * 100
missing_info = pd.concat([missing_count,missing_percent],axis=1,keys=['Count','Percent']).sort_values(by='Percent',ascending=False)
missing_info['dtype'] = all_data.dtypes[  missing_info.index ]
missing_info

,Count,Percent,dtype
PoolQC,2908,99.691464,object
MiscFeature,2812,96.400411,object
Alley,2719,93.212204,object
Fence,2346,80.425094,object
FireplaceQu,1420,48.680151,object
LotFrontage,486,16.660953,float64
GarageFinish,159,5.450806,object
GarageQual,159,5.450806,object
GarageCond,159,5.450806,object
GarageYrBlt,159,5.450806,float64


In [145]:
for col in missing_info.index:
    missing_info.loc[col,'mode_percent'] = all_data[col].value_counts().sort_values(ascending=False).values[0] / len(all_data) * 100

In [146]:
missing_info

,Count,Percent,dtype,mode_percent
PoolQC,2908,99.691464,object,0.137127
MiscFeature,2812,96.400411,object,3.256771
Alley,2719,93.212204,object,4.113816
Fence,2346,80.425094,object,11.278711
FireplaceQu,1420,48.680151,object,25.437093
LotFrontage,486,16.660953,float64,9.461776
GarageFinish,159,5.450806,object,42.166610
GarageQual,159,5.450806,object,89.201234
GarageCond,159,5.450806,object,90.915324
GarageYrBlt,159,5.450806,float64,4.868015


In [147]:
num_feats = all_data.select_dtypes(exclude='object')
cat_feats = all_data.select_dtypes(include='object')

In [148]:
missing_num_feats = []
missing_cat_feats = []
for col in missing_info.index:
    if col in num_feats:
        missing_num_feats.append(col)
    if col in cat_feats:
        missing_cat_feats.append(col)

In [149]:
missing_num_feats

['LotFrontage',
 'GarageYrBlt',
 'MasVnrArea',
 'BsmtFullBath',
 'BsmtHalfBath',
 'GarageArea',
 'GarageCars',
 'TotalBsmtSF',
 'BsmtUnfSF',
 'BsmtFinSF2',
 'BsmtFinSF1']

In [150]:
missing_cat_feats

['PoolQC',
 'MiscFeature',
 'Alley',
 'Fence',
 'FireplaceQu',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'GarageType',
 'BsmtExposure',
 'BsmtCond',
 'BsmtQual',
 'BsmtFinType2',
 'BsmtFinType1',
 'MasVnrType',
 'MSZoning',
 'Functional',
 'Utilities',
 'Electrical',
 'KitchenQual',
 'Exterior2nd',
 'Exterior1st',
 'SaleType']

In [152]:
all_data["PoolQC"] = all_data["PoolQC"].fillna("None")
all_data["MiscFeature"] = all_data["MiscFeature"].fillna("None")
all_data["Alley"] = all_data["Alley"].fillna("None")
all_data["Fence"] = all_data["Fence"].fillna("None")
all_data["FireplaceQu"] = all_data["FireplaceQu"].fillna("None")

In [153]:
all_data.groupby('Neighborhood')['LotFrontage'].median()

Neighborhood
Blmngtn    43.0
Blueste    24.0
BrDale     21.0
BrkSide    51.0
ClearCr    80.5
CollgCr    70.0
Crawfor    70.0
Edwards    64.5
Gilbert    64.0
IDOTRR     60.0
MeadowV    21.0
Mitchel    74.0
NAmes      73.0
NPkVill    24.0
NWAmes     80.0
NoRidge    89.0
NridgHt    92.0
OldTown    60.0
SWISU      60.0
Sawyer     72.0
SawyerW    67.0
Somerst    72.5
StoneBr    60.0
Timber     82.0
Veenker    80.0
Name: LotFrontage, dtype: float64

In [161]:
all_data.loc[all_data['LotFrontage'].isnull() , 'LotFrontage'] = all_data[all_data['LotFrontage'].isnull()].apply(lambda row : all_data[all_data['Neighborhood'] == row['Neighborhood']].median() , axis=1)

In [164]:
all_data['LotFrontage'].isnull().sum()

0

In [165]:
for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'):
    all_data[col] = all_data[col].fillna('None')

In [166]:
for col in ('GarageYrBlt', 'GarageArea', 'GarageCars'):
    all_data[col] = all_data[col].fillna(0)

In [167]:
for col in ('BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
    all_data[col] = all_data[col].fillna(0)

In [168]:
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    all_data[col] = all_data[col].fillna('None')

In [169]:
all_data["MasVnrType"] = all_data["MasVnrType"].fillna("None")
all_data["MasVnrArea"] = all_data["MasVnrArea"].fillna(0)

In [174]:
all_data['MSZoning'] = all_data['MSZoning'].fillna(all_data['MSZoning'].mode()[0])

In [175]:
all_data = all_data.drop(['Utilities'], axis=1)

In [176]:
all_data["Functional"] = all_data["Functional"].fillna("Typ")

In [177]:
all_data['Electrical'] = all_data['Electrical'].fillna(all_data['Electrical'].mode()[0])

In [178]:
all_data['KitchenQual'] = all_data['KitchenQual'].fillna(all_data['KitchenQual'].mode()[0])

In [179]:
all_data['Exterior1st'] = all_data['Exterior1st'].fillna(all_data['Exterior1st'].mode()[0])
all_data['Exterior2nd'] = all_data['Exterior2nd'].fillna(all_data['Exterior2nd'].mode()[0])

In [180]:
all_data['SaleType'] = all_data['SaleType'].fillna(all_data['SaleType'].mode()[0])all_data['SaleType'] = all_data['SaleType'].fillna(all_data['SaleType'].mode()[0])

In [186]:
all_data.isnull().sum().sum()

0